In [3]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from InstructorEmbedding import INSTRUCTOR

/Users/crys/miniconda3/envs/htx_app/lib/python3.9/site-packages/InstructorEmbedding/instructor.py:7: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import trange


In [4]:
model = INSTRUCTOR('hkunlp/instructor-large')

load INSTRUCTOR_Transformer
max_seq_length  512


/Users/crys/miniconda3/envs/htx_app/lib/python3.9/site-packages/sentence_transformers/models/Dense.py:63: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict

In [5]:
cv_valid_fpath = "../../asr/processed-cv-valid-dev.csv"
hotword_list = ['be careful', 'destroy' 'stranger']

In [6]:
valid_df = pd.read_csv(cv_valid_fpath)
valid_df.head(3)

,filename,text,up_votes,down_votes,age,gender,accent,duration,group_type,generated_text
0,../data/raw/common_voice/cv-valid-dev/cv-valid...,and in that way the months passed,1,0,NaN,NaN,NaN,NaN,cv-valid-dev,AND IN THAT WAY THE MONTHS PASSED
1,../data/raw/common_voice/cv-valid-dev/cv-valid...,about time one of you lunkheads said it,1,0,NaN,NaN,NaN,NaN,cv-valid-dev,ABRAT TIME ONE OF EULANCATS SAID IT
2,../data/raw/common_voice/cv-valid-dev/cv-valid...,the burning fire had been extinguished,3,1,NaN,NaN,NaN,NaN,cv-valid-dev,THE BONIPOYRE HAD BEEN EXTINGUISHED


In [7]:
unq_valid_df = valid_df.drop_duplicates(subset=['text'])

In [8]:
hot_phrases = valid_df[(valid_df['text'].str.contains("be careful")) | (valid_df['text'].str.contains("destroy")) | (valid_df['text'].str.contains("stranger"))]
hot_phrases.head(5)

,filename,text,up_votes,down_votes,age,gender,accent,duration,group_type,generated_text
901,../data/raw/common_voice/cv-valid-dev/cv-valid...,be careful with your prognostications said the...,1,0,NaN,NaN,NaN,NaN,cv-valid-dev,BE CAREFUL THAT YOU PROGNOSTICATIONS SAID THE ...
905,../data/raw/common_voice/cv-valid-dev/cv-valid...,the stranger was speaking of things that very ...,2,1,NaN,NaN,NaN,NaN,cv-valid-dev,THE STRANGER WAS SPEAKING OF THINGS THAT PRETT...


In [9]:
hot_phrases['embeddings'] = hot_phrases['text'].apply(lambda x: model.encode([["Calculate sentence similarities:", x]]))

/var/folders/sq/z4mv25d17y3_fdz56x8p97nr0000gn/T/ipykernel_11411/3234989317.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hot_phrases['embeddings'] = hot_phrases['text'].apply(lambda x: model.encode([["Calculate sentence similarities:", x]]))


In [10]:
hot_phrases.head(3)

,filename,text,up_votes,down_votes,age,gender,accent,duration,group_type,generated_text,embeddings
901,../data/raw/common_voice/cv-valid-dev/cv-valid...,be careful with your prognostications said the...,1,0,NaN,NaN,NaN,NaN,cv-valid-dev,BE CAREFUL THAT YOU PROGNOSTICATIONS SAID THE ...,"[[-0.033820048, -0.02761182, -0.036558174, 0.0..."
905,../data/raw/common_voice/cv-valid-dev/cv-valid...,the stranger was speaking of things that very ...,2,1,NaN,NaN,NaN,NaN,cv-valid-dev,THE STRANGER WAS SPEAKING OF THINGS THAT PRETT...,"[[-0.036920454, -0.017686976, -0.018310416, -0..."


In [19]:
is_hot = []
for _ , row in unq_valid_df.iterrows():
    curr_embeddings = model.encode([["Calculate sentence similarities:", row['text']]])
    match = False
    for _ , hot_row in hot_phrases.iterrows():
        similarities = cosine_similarity(curr_embeddings,hot_row['embeddings'])
        if similarities[0][0] >= 0.85:
            print(f"{row['text']} is similar to {hot_row['text']}")
            is_hot += [True]
            match = True
            break
    if not match:
        is_hot += [False]

dont share your opinions so readily because you never know who is listening is similar to be careful with your prognostications said the stranger
don't give in to your fears said the alchemist in a strangely gentle voice is similar to be careful with your prognostications said the stranger
in those days very few of the people had any idea about astronomy is similar to the stranger was speaking of things that very few people knew about
i make my living forecasting the future for people he said is similar to be careful with your prognostications said the stranger
in a way so am i he said is similar to be careful with your prognostications said the stranger
what did i tell you is similar to be careful with your prognostications said the stranger
maybe this is an omen said the englishman half aloud is similar to be careful with your prognostications said the stranger
can i help you asked the man behind the window is similar to be careful with your prognostications said the stranger
learn t

In [21]:
unq_valid_df["similarity"] = is_hot
unq_valid_df.to_csv("cv-valid-dev.csv", index=False)

/var/folders/sq/z4mv25d17y3_fdz56x8p97nr0000gn/T/ipykernel_11411/2322071121.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unq_valid_df["similarity"] = is_hot
